Our mission for this project is to scrap sevaral newspaper by a keyword.
Here we choose Banking, Economy and Football keyword. By inputing those keyword, we will scrap news related those keyword from different newspapers automaticly. For this project, we choose Daily Star and Prothom Alo newspaper. After scraping those news, it will store as a csv file where we will have the headline of news, link for the news, publishing time.

N.B: This project is based on 15 July, 2024 script. This script may not work later.

Importing Library

In [66]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
from datetime import datetime, timedelta

In [67]:
x=input('Category=')
print(f"Wait few seconds for {x} category news")

if x=="Banking":
    # Initialize the WebDriver
    driver = webdriver.Chrome()
    driver.get("https://www.thedailystar.net/business/banking")
    time.sleep(2)
    
    
    # Define the XPath for the articles
    article_xpath = '//div[@class="columns small-12 medium-3 large-3"]'
    
    # Wait for the articles to be present
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, article_xpath)))
    
    # List to hold the data
    data = []
    
    # Extract data
    articles = driver.find_elements(By.XPATH, article_xpath)
    for article in articles:
        try:
            headline_element = article.find_element(By.XPATH, './/h3[@class="title"]/a')
            headline = headline_element.text
            link = headline_element.get_attribute('href')
            time_element = article.find_element(By.XPATH, './/div[@class="card-top color-iron text-12"]/span[@class="interval"]')
            time_text = time_element.text
            print(f"Time Text: {time_text}")
            paper = "Daily Star"
            category="Economy"
            if re.search(r'\d{1,2}[hm] ago', time_text):
            # Calculate the published time
                current_time = datetime.now()
                if 'h' in time_text:
                    hours_ago = int(re.search(r'\d{1,2}', time_text).group())
                    published_time = current_time - timedelta(hours=hours_ago)
                elif 'm' in time_text:
                    minutes_ago = int(re.search(r'\d{1,2}', time_text).group())
                    published_time = current_time - timedelta(minutes=minutes_ago)
                    
                published_time_str = published_time.strftime('%I:%M %p')
                published_date_str = published_time.strftime('%d/%m/%Y')
                
                data.append({'Headline': headline, 'Link': link, 'Time': time_text, 'Published Time': published_time_str, 
                             'Published Date': published_date_str, 'Source': paper, 'Category': category})
        except Exception as e:
            print(f"Error extracting data from article: {e}")
    
    # Close the WebDriver
    driver.quit()
    
    # Create a DataFrame and save it to a CSV file
    df = pd.DataFrame(data)
    df.to_csv('banking_headlines.csv', index=False)
    
    print(df.head(20))
    
elif x=="Economy":
    # Initialize the WebDriver
    driver = webdriver.Chrome()

    data = []
    
    def extract_from_daily_star():
        driver.get("https://www.thedailystar.net/business/economy")
        time.sleep(2)
        
        # Define the XPath for the articles
        article_xpath = '//div[@class="columns small-12 medium-3 large-3"]'
        
        # Wait for the articles to be present
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, article_xpath)))
        
        # Extract data
        articles = driver.find_elements(By.XPATH, article_xpath)
        for article in articles:
            try:
                headline_element = article.find_element(By.XPATH, './/h3[@class="title"]/a')
                headline = headline_element.text
                link = headline_element.get_attribute('href')
                time_element = article.find_element(By.XPATH, './/div[@class="card-top color-iron text-12"]/span[@class="interval"]')
                time_text = time_element.text
                paper = "Daily Star"
                category="Economy"
                if re.search(r'\d{1,2}[hm] ago', time_text):
                    # Calculate the published time
                    current_time = datetime.now()
                    if 'h' in time_text:
                        hours_ago = int(re.search(r'\d{1,2}', time_text).group())
                        published_time = current_time - timedelta(hours=hours_ago)
                    elif 'm' in time_text:
                        minutes_ago = int(re.search(r'\d{1,2}', time_text).group())
                        published_time = current_time - timedelta(minutes=minutes_ago)
                    
                    published_time_str = published_time.strftime('%I:%M %p')
                    published_date_str = published_time.strftime('%d/%m/%Y')
                    
                    data.append({'Headline': headline, 'Link': link, 'Time': time_text, 'Published Time': published_time_str, 
                                 'Published Date': published_date_str, 'Source': paper, 'Category': category})
            except Exception as e:
                print(f"Error extracting data from article: {e}")
    
    def extract_from_prothom_alo():
        driver.get("https://en.prothomalo.com/business/local")
        time.sleep(2)
        
        try:
            # Adjust the wait time and the locator as needed
            close_button = WebDriverWait(driver, 20).until(
                EC.visibility_of_element_located((By.XPATH, '//*[@id="pushengage-opt-in-1-close"]'))
            )
            close_button.click()
            print("Pop-up closed.")
        except:
            print("No pop-up found or couldn't close the pop-up.")
        
        # Define the XPath for the articles
        article_xpath = '//div[contains(@class,"xkXol")]'
                
        # Wait for the articles to be present
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, article_xpath)))
        
        # Extract data
        articles = driver.find_elements(By.XPATH, article_xpath)
        for article in articles:
            try:
                headline_element = article.find_element(By.XPATH, './/h3/a')
                headline = headline_element.text
                link = headline_element.get_attribute('href')
                time_element = article.find_element(By.XPATH, './/time')
                time_text = time_element.text
                paper = "Prothom Alo"
                category="Economy"
                if re.search(r'\d{1,2}\s*(hours|minutes|hm) ago', time_text):
                    # Calculate the published time
                    current_time = datetime.now()
                    if 'hours' in time_text or 'h' in time_text:
                        hours_ago = int(re.search(r'\d{1,2}', time_text).group())
                        published_time = current_time - timedelta(hours=hours_ago)
                    elif 'minutes' in time_text or 'm' in time_text:
                        minutes_ago = int(re.search(r'\d{1,2}', time_text).group())
                        published_time = current_time - timedelta(minutes=minutes_ago)
                    
                    published_time_str = published_time.strftime('%I:%M %p')
                    published_date_str = published_time.strftime('%d/%m/%Y')
                    
                    data.append({'Headline': headline, 'Link': link, 'Time': time_text, 'Published Time': published_time_str, 
                                 'Published Date': published_date_str, 'Source': paper, 'Category': category})
            except Exception as e:
                print(f"Error extracting data from article: {e}")
    
    # Extract data from both websites
    extract_from_daily_star()
    extract_from_prothom_alo()
    
    # Close the WebDriver
    driver.quit()
    
    # Create a DataFrame and save it to a CSV file
    df = pd.DataFrame(data)
    df.to_csv('economy_headlines.csv', index=False)
    
    print(df.head(20))

elif x=="Football":
    # Initialize the WebDriver
    driver = webdriver.Chrome()

    data = []
    
    def extract_from_daily_star():
        driver.get("https://www.thedailystar.net/sports/football")
        time.sleep(2)
        
        # Define the XPath for the articles
        article_xpath = '//div[@class="columns small-12 medium-3 large-3"]'
        
        # Wait for the articles to be present
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, article_xpath)))
        
        # Extract data
        articles = driver.find_elements(By.XPATH, article_xpath)
        for article in articles:
            try:
                headline_element = article.find_element(By.XPATH, './/h3[@class="title"]/a')
                headline = headline_element.text
                link = headline_element.get_attribute('href')
                time_element = article.find_element(By.XPATH, './/div[@class="card-top color-iron text-12"]/span[@class="interval"]')
                time_text = time_element.text
                paper = "Daily Star"
                category="Football"
                if re.search(r'\d{1,2}[hm] ago', time_text):
                    # Calculate the published time
                    current_time = datetime.now()
                    if 'h' in time_text:
                        hours_ago = int(re.search(r'\d{1,2}', time_text).group())
                        published_time = current_time - timedelta(hours=hours_ago)
                    elif 'm' in time_text:
                        minutes_ago = int(re.search(r'\d{1,2}', time_text).group())
                        published_time = current_time - timedelta(minutes=minutes_ago)
                    
                    published_time_str = published_time.strftime('%I:%M %p')
                    published_date_str = published_time.strftime('%d/%m/%Y')
                    
                    data.append({'Headline': headline, 'Link': link, 'Time': time_text, 'Published Time': published_time_str, 
                                 'Published Date': published_date_str, 'Source': paper, 'Category': category})
            except Exception as e:
                print(f"Error extracting data from article: {e}")
    
    def extract_from_prothom_alo():
        driver.get("https://en.prothomalo.com/sports/football")
        time.sleep(2)
        
        try:
            # Adjust the wait time and the locator as needed
            close_button = WebDriverWait(driver, 20).until(
                EC.visibility_of_element_located((By.XPATH, '//*[@id="pushengage-opt-in-1-close"]'))
            )
            close_button.click()
            print("Pop-up closed.")
        except:
            print("No pop-up found or couldn't close the pop-up.")
        
        # Define the XPath for the articles
        article_xpath = '//div[contains(@class,"xkXol")]'
                
        # Wait for the articles to be present
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, article_xpath)))
        
        # Extract data
        articles = driver.find_elements(By.XPATH, article_xpath)
        for article in articles:
            try:
                headline_element = article.find_element(By.XPATH, './/h3/a')
                headline = headline_element.text
                link = headline_element.get_attribute('href')
                time_element = article.find_element(By.XPATH, './/time')
                time_text = time_element.text
                paper = "Prothom Alo"
                category="Football"
                if re.search(r'\d{1,2}\s*(hours|minutes|hm) ago', time_text):
                    # Calculate the published time
                    current_time = datetime.now()
                    if 'hours' in time_text or 'h' in time_text:
                        hours_ago = int(re.search(r'\d{1,2}', time_text).group())
                        published_time = current_time - timedelta(hours=hours_ago)
                    elif 'minutes' in time_text or 'm' in time_text:
                        minutes_ago = int(re.search(r'\d{1,2}', time_text).group())
                        published_time = current_time - timedelta(minutes=minutes_ago)
                    
                    published_time_str = published_time.strftime('%I:%M %p')
                    published_date_str = published_time.strftime('%d/%m/%Y')
                    
                    data.append({'Headline': headline, 'Link': link, 'Time': time_text, 'Published Time': published_time_str, 
                                 'Published Date': published_date_str, 'Source': paper, 'Category': category})
            except Exception as e:
                print(f"Error extracting data from article: {e}")
    
    # Extract data from both websites
    extract_from_daily_star()
    extract_from_prothom_alo()
    
    # Close the WebDriver
    driver.quit()
    
    # Create a DataFrame and save it to a CSV file
    df = pd.DataFrame(data)
    df.to_csv('football_headlines.csv', index=False)
    
    print(df.head(20))
    
else:
    print("Please write correct category. N.B: Banking, Economy, Football")

Category= Football


Wait few seconds for Football category news
Pop-up closed.
                                             Headline  \
0   Joy, heartbreak splayed across newspapers afte...   
1            Messi-blessed Yamal breaks Pele’s record   
2                  Thomas Mueller ends Germany career   
3   "Fine margins" as Southgate's England miss out...   
4   Copa triumph takes Messi past every footballer...   
5   Scaloni hails Argentina for 'Triple Crown' tri...   
6   Joy in the streets of Argentina after Copa vic...   
7   Tired Colombia were impacted by delayed start ...   
8   Argentina defeat Colombia to win record 16th C...   
9                Messi goes off injured in Copa final   
10  Spain beat England to win Euro 2024 final with...   
11  Crowd chaos as fans kept waiting outside Copa ...   

                                                 Link         Time  \
0   https://www.thedailystar.net/sports/sports-spe...      16m ago   
1   https://www.thedailystar.net/sports/sports-spe...      